In [51]:
import pandas as pd

# Sequence Similarity Demo
In this demo, we will answer the question:

_How does the primary sequence of TMPRSS2 differ between species that one would encounter in a farm environment?_

We will address this question using sequence alignment and analysis tools from the [Biopython](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec81) Python library.

## Outline

* Using the [Biopython tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec385) as reference
* Prerequisites
    * Reading on PSSMs: [Rice.edu](https://www.cs.rice.edu/~ogilvie/comp571/2018/09/11/pssm.html)

### Part 1: Preparing input sequences

* Intro to `Bio.Align`
* Learn how to filter sequence records in a multiple sequence alignment by:
    * Species name
    * Sequence snippets
* Find the 
* Generate consensus sequences for the cat sequence

### Part 2: Analyzing aligned sequences

* Compare human homolog to the mouse
    * Compute a [log odds substitution matrix](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec390)
    * What are the log odds of the following polymorphisms?
        * Hydrophobic -> hydrophilic and vice versa
        * Aromatic -> non-aromatic and vice versa
    * Construct a more generalized PSSM for the above categories of "penalized polymorphisms"
        * For instance, we want to parse `R -> Y` and `S -> I` to `hydrophilic -> hydrophobic`

## "Homework"
"Homework" is a recommendedation. If you find yourself more interested in a different analysis, say in a comparison of variants **within** Homo sapiens, feel free to do that analysis instead.

* Repeat analysis for each of the other domestic species (dog, horse, chicken, etc.)
* Generate a "generalized PSSM" for the other types of penalized polymorphisms, such as `acidic -> basic`, `bulky -> small`, `aromatic -> non-aromatic`, etc.

We're using [Biopython](http://biopython.org/DIST/docs/tutorial/Tutorial.html) again. If the below import commands fail, you might need to install Biopython from the command line:
```bash
pip install biopython

# or using poetry
poetry add biopython
```

In [125]:
from Bio.Align import AlignInfo, MultipleSeqAlignment
from Bio import AlignIO, Alphabet, SeqRecord, Seq

# Part #1
____

# Read the alignment records
We use the Python function from the Biopython package: `Bio.AlignIO.read` to read the trimmed alignment file. This Python function reads the `*.txt` file in the `'fasta'` format and returns an instance of `Bio.Align.MultipleSeqAlignment` (documentation can be found [here](https://biopython.org/DIST/docs/api/Bio.Align.MultipleSeqAlignment-class.html) and [here](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec81)).

In [53]:
alignment = AlignIO.read(open('./trimmed_alg.txt'), format='fasta')
alignment

<<class 'Bio.Align.MultipleSeqAlignment'> instance (9757 records of length 60918, SingleLetterAlphabet()) at 7f683a84b110>

Each element of this list-like instance is a sequence:

In [54]:
alignment[0] 

SeqRecord(seq=Seq('------------------------------------------------------...---', SingleLetterAlphabet()), id='7260.FBpp0240705', name='7260.FBpp0240705', description='7260.FBpp0240705', dbxrefs=[])

This instance of `Bio.Align.MultipleSeqAlignment` is a lot like a Python list. For instance, you can:

In [55]:
# get the number of sequences in this alignment
print("number of sequence records: ", len(alignment))

# iterate over the sequence records in the alignment
record_counter = 0
for record in alignment:
    record_counter += 1
print("number of sequence records (a different way): ", record_counter)

# get the 100th sequence record in the alignment
print("ID of the 100th sequence: ", alignment[99].id)

number of sequence records:  9757
number of sequence records (a different way):  9757
ID of the 100th sequence:  9796.ENSECAP00000016722


# Filter the sequences in the alignment
For now, we're only interested in "domestic species," or species whose scientific name is in the Python list `domestic_sp_names`:

In [56]:
domestic_sp_names = [
    'Homo sapiens', # human
    'Mus musculus', # mouse
    'Canis lupus familiaris', # dog
    'Felis catus', # cat
    'Bos taurus', # cattle
    'Equus caballus', # horse
    'Gallus gallus' # chicken
]

The sequences in the `Bio.Align.MultipleSeqAlignment` are for **all** the species that EggNOG could find, including worms, polar bears, and other species that we're not interested in.

Let's filter out sequences from species whose names are **not** in the list `domestic_sp_names`. To do this, we will:
1. Get the scientific name for each species, and load it into the `description` attribute of each sequence. This should be familiar from the [descriptive stats demo](../descriptive_stats_demo/eggNOG_alignment_metadata.ipynb).
2. Use a [list comprehension](https://github.com/wilfredinni/python-cheatsheet#list-comprehension) to get the list of sequences for species that we are interested in.
3. The above step will generate a Python list; it will need to be converted to an instance of  `Bio.Align.MultipleSeqAlignment` if we want to use fancy Biopython analysis tools on it.

## Step #1: Get scientific name for each species
This should be familiar from the [descriptive stats demo](../descriptive_stats_demo/eggNOG_alignment_metadata.ipynb).

In [57]:
tmprss2_ext = pd.read_table('../descriptive_stats_demo/extended_members.txt', header=None)
tmprss2_ext.columns = ['id_1', 'id_2', 'species', '', '']
# tmprss2_ext.head()

In [58]:
for record in alignment:
    
    # while we're at it, let's make sure that Biopython knows these
    # are protein sequences
    record.seq.alphabet = Alphabet.generic_protein
    
    # from visual inspection we know the name format is XXXX.unique_id,
    # so we split on "." and take the last element of the list
    id_code = record.id.split('.')[-1]
    
    # reference the metadata to get the species name
    sp_name = tmprss2_ext[tmprss2_ext['id_1'] == id_code]['species'].values
    
    try:
        sp_name = sp_name.item()
    except ValueError:
        sp_name = None
    
    # assign the species name to the species attribute
    record.description = sp_name

## Step #2: Use a list comprehension to filter to domestic species

In [59]:
dom_aln_list = [record for record in alignment
                if record.description in domestic_sp_names]

We see that the length of this filtered list is much shorter:

In [60]:
print("number of records for all species:", len(alignment))
print("number of records for domestic species:", len(dom_aln_list))

number of records for all species: 9757
number of records for domestic species: 732


## Step #3: Convert this list to a new `MultipleSeqAlignment` instance

In [61]:
dom_aln = MultipleSeqAlignment(dom_aln_list)

`dom_aln` has the same data, but is a different type of Python variable:

In [62]:
print("dom_aln_list is type:", type(dom_aln_list))
print("dom_aln is type:", type(dom_aln))

dom_aln_list is type: <class 'list'>
dom_aln is type: <class 'Bio.Align.MultipleSeqAlignment'>


# Get the sequence of human TMPRSS2
Before we start comparing sequences to each other, let's get the sequence of TMPRSS2 in `Homo sapiens`. This is the sequence that we will compare other species' homologs to.

To do this filtering, let's use a list comprehension, then convert to a `MultipleSeqAlignment`, just like we did before:

In [63]:
human_aln_list = [
    record for record in dom_aln
    if record.description == 'Homo sapiens'
]
human_aln = MultipleSeqAlignment(human_aln_list)

We see that there are many records in the alignment that have `Homo sapiens` as the species:

In [64]:
len(human_aln)

118

It would be interesting to look at how the differences between these 118 variants _within_ the human species, but let's move on to our inter-species analysis for this demo.

## Get the sequence of human isoform 1

Let's find the sequence record that has the same sequence as isoform 1 on the [TMPRSS2 UniProt page](https://www.uniprot.org/uniprot/O15393#O15393-1). The first few residues of this isoform are `MPPAPPGG`:

In [69]:
isoform_aln_list = [
    record for record in human_aln
    if 'MPPAPPGG' in str(record.seq).replace("-", "")
]

In [70]:
print("number of human sequences that contain MPPAPPGG:", len(isoform_aln_list))
human_iso1 = isoform_aln_list[0]
human_iso1

number of human sequences that contain MPPAPPGG: 1


SeqRecord(seq=Seq('------------------------------------------------------...---', ProteinAlphabet()), id='9606.ENSP00000381588', name='9606.ENSP00000381588', description='Homo sapiens', dbxrefs=[])

This is an aligned sequence, so it has a lot of `-` characters that signify residues that are missing relative to other sequences in `alignment`:

In [71]:
str(human_iso1.seq)

'---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

We can remove these characters using Python's string replacement method, allowing us to more easily look at the amino acid sequence:

In [72]:
str(human_iso1.seq).replace('-', '')

'MPPAPPGGESGCEERGAAGHIEHSRYLSLLDAVDNSKMALNSGSPPAIGPYYENHGYQPENPYPAQPTVVPTVYEVHPAQYYPSPVPQYAPRVLTQASNPVVCTQPKSPSGTVCTSKTKKALCITLTLGTFLVGAALAAGLLWKFMGSKCSNSGIECDSSGTCINPSNWCDGVSHCPGGEDENRCVRLYGPNFILQVYSSQRKSWHPVCQDDWNENYGRAACRDMGYKNNFYSSQGIVDDSGSTSFMKLNTSAGNVDIYKKLYHSDACSSKAVVSLRCIACGVNLNSSRQSRIVGGESALPGAWPWQVSLHVQNVHVCGGSIITPEWIVTAAHCVEKPLNNPWHWTAFAGILRQSFMFYGAGYQVEKVISHPNYDSKTKNNDIALMKLQKPLTFNDLVKPVCLPNPGMMLQPEQLCWISGWGATEEKGKTSEVLNAAKVLLIETQRCNSRYVYDNLITPAMICAGFLQGNVDSCQGDSGGPLVTSKNNIWWLIGDTSWGSGCAKAYRPGVYGNVMVFTDWIYRQMRADG'

We also notice that most of the sequence of interest is in the middle of the aligned sequence. Let's trim the aligned sequence to generate a compact aligned sequence that it starts with `MPPAPP` and ends with `ADG`. To do this, we will make use of the [`str.index`](https://docs.python.org/2/library/stdtypes.html?highlight=index#str.index) method:

In [81]:
index_nterm = str(human_iso1.seq).index('MPPAPP')
index_cterm = str(human_iso1.seq).index('ADG')

# since we want to cut at ADG^, not ^ADG, we add 3 characters to this index
index_cterm += 3

print("index of N-terminus:", index_nterm)
print("index of C-terminus:", index_cterm)

index of N-terminus: 33713
index of C-terminus: 38856


We can use these indices to trim to the compact sequence:

In [82]:
human_compact = human_iso1[index_nterm:index_cterm]
str(human_compact.seq)

'MPPAPP----------GGESG-CEE-----------------------------------------------------------R--G-A-A---------GHIEHSRYLS-L-LD---------AV-D----------N----SK-------------------------------------------------------M------------------------------------------------------------------------ALNSG-------------------------S----------P----------------------------------------------------------P---AI---G----P-Y---YENHG----------------------------------------YQPE---------NPY-----------------------------------------------------------------------------------------------------------------------------------------P-A-Q-----------------PT----------------VV-P------------------------------------------------------T----------------------------------V-------------------------------------YE-V-H---P----------A------------------------------QY--Y----P-S--------------P-V-P------Q----YAPRVLT---Q-A--SN--P-------V----V--CTQ--PK-SP---SG-----T---------------------------------------------------------------------------------------

These N-terminus and C-terminus indices will be useful when we want to trim sequence records for other species.

# Generate consensus sequences for cat homolog

Just like the sequence records for `Homo sapiens`, the records for the other `domestic_sp_names` have duplicates. For example, let's look at `Felis catus`, or domesticated cat:

In [84]:
cat_aln_list = [
    record for record in dom_aln
    if record.description == 'Felis catus'
]
cat_aln = MultipleSeqAlignment(cat_aln_list)

In [86]:
len(cat_aln)

100

Let's compare 1 sequence, instead of all 100 variants, of cat homolog to the human homolog. To do this, we will generate a **consensus sequence** ([Wikipedia](https://en.wikipedia.org/wiki/Consensus_sequence#:~:text=In%20molecular%20biology%20and%20bioinformatics,position%20in%20a%20sequence%20alignment.)) for the cat variants. We do this in 2 steps:
1. Generate a `Bio.Align.AlignInfo.SummaryInfo` instance from the `MultipleSeqAlignment`
2. Call the `SummaryInfo` method `dumb_consensus`, which runs a very simple consensus sequence finding algorithm.

## Step #1

In [87]:
cat_aln_summary = AlignInfo.SummaryInfo(cat_aln)
cat_aln_summary

## Step #2

In [109]:
cat_aln_consensus = cat_aln_summary.dumb_consensus()
cat_aln_consensus

Seq('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...XXX', ProteinAlphabet())

Let's use the N-terminus and C-terminus locations that we calculated above to compact this consensus sequence:

In [120]:
cat_consensus_compact = cat_aln_consensus[index_nterm:index_cterm]
str(cat_consensus_compact).replace('X', '-')

'---------E-----A--------------------AL-L---------------------------------------RG------G--------QEG-T-------------P---------------P--S----L--------YE-----------------------------------------------G------------A------------H-----IPC---EL-----------C-----------------P-P-H-------C---------P------------M---G-------------------L------------------TP-----------------------------T---AA---H----C----S--------------------GR----EGRRLEME--RRR------------F-------V---L--------------------EC-----------PTD-----C---------I---------H---V------G--A-A------PN----------AG-------------------------------VHL---E-----------------------MLE-----------------------A-------------C-------------H-P------LH---------------------W---L---------P------I----------S------------------W-------D---LL-----ACLLPPEPGAL-------G----------K---------------YKGE---------------------------------D-V--------------L-----------------------P----E--G--------------------------------------------------------------------------LP------------------

Finally, this consensus sequence is a `Seq`, not a `SeqRecord`. Let's convert it to a `SeqRecord` so we can compare it to the human sequence:

In [128]:
# convert 'X' to '-' for consistency with human sequence
# and convert to a Seq.Seq instance
cat_replaced_str =  str(cat_consensus_compact).replace('X', '-')
cat_consensus_replaced = Seq.Seq(cat_replaced_str)

# then convert to a SeqRecord.SeqRecord instance
cat_record_compact = SeqRecord.SeqRecord(cat_consensus_replaced, description='Felis catus', name='dumb_consensus')
cat_record_compact

SeqRecord(seq=Seq('---------E-----A--------------------AL-L--------------...---'), id='<unknown id>', name='dumb_consensus', description='Felis catus', dbxrefs=[])

# Part 2: the fun stuff
**Finally**, we have human TMPRSS2 and a consensus sequence for cat TMPRSS2. The sequences are aligned and ready for some more advanced analysis with the help of Biopython.

Let's start looking at ways we can compare the two sequences. To start, we will answer the question:

At every location in the sequence, what is the percent probability that this position will be a tyrosine (Y), leucine (L), or any other amino acid?

To do this, we will calculate a [**position specific score matrix**](https://www.cs.rice.edu/~ogilvie/comp571/2018/09/11/pssm.html) (PSSM). Let's generate a new, very short `MultipleSeqAlignment` between our human and cat sequences:

In [137]:
hum_cat_aln = MultipleSeqAlignment([human_compact, cat_record_compact])
hum_cat_aln

<<class 'Bio.Align.MultipleSeqAlignment'> instance (2 records of length 5143, Alphabet()) at 7f682fa1a0d0>

Now we can generate a `SummaryInfo` instance like we did before, and calculate the PSSM:

In [139]:
hum_cat_summary = AlignInfo.SummaryInfo(hum_cat_aln)
hum_cat_summary

In [164]:
hum_cat_pssm = hum_cat_summary.pos_specific_score_matrix(human_compact)
hum_cat_pssm

We can look at the data in the PSSM by inspecting the `pssm` attribute.

The PSSM is a Python list, where each element is a [tuple](https://github.com/wilfredinni/python-cheatsheet#tuple-data-type) of length 2. The first element of the tuple is the amino acid in the human sequence, and the second element is a Python [dictionary](https://github.com/wilfredinni/python-cheatsheet#dictionaries-and-structuring-data). The dictionary keys are all the naturally occurring amino acids, and the values are the number of times that amino acid was found at that position in the alignment.

In [165]:
hum_cat_pssm.pssm

[('M',
  {'-': 1.0,
   'A': 0,
   'C': 0,
   'D': 0,
   'E': 0,
   'F': 0,
   'G': 0,
   'H': 0,
   'I': 0,
   'K': 0,
   'L': 0,
   'M': 1.0,
   'N': 0,
   'P': 0,
   'Q': 0,
   'R': 0,
   'S': 0,
   'T': 0,
   'V': 0,
   'W': 0,
   'Y': 0}),
 ('P',
  {'-': 1.0,
   'A': 0,
   'C': 0,
   'D': 0,
   'E': 0,
   'F': 0,
   'G': 0,
   'H': 0,
   'I': 0,
   'K': 0,
   'L': 0,
   'M': 0,
   'N': 0,
   'P': 1.0,
   'Q': 0,
   'R': 0,
   'S': 0,
   'T': 0,
   'V': 0,
   'W': 0,
   'Y': 0}),
 ('P',
  {'-': 1.0,
   'A': 0,
   'C': 0,
   'D': 0,
   'E': 0,
   'F': 0,
   'G': 0,
   'H': 0,
   'I': 0,
   'K': 0,
   'L': 0,
   'M': 0,
   'N': 0,
   'P': 1.0,
   'Q': 0,
   'R': 0,
   'S': 0,
   'T': 0,
   'V': 0,
   'W': 0,
   'Y': 0}),
 ('A',
  {'-': 1.0,
   'A': 1.0,
   'C': 0,
   'D': 0,
   'E': 0,
   'F': 0,
   'G': 0,
   'H': 0,
   'I': 0,
   'K': 0,
   'L': 0,
   'M': 0,
   'N': 0,
   'P': 0,
   'Q': 0,
   'R': 0,
   'S': 0,
   'T': 0,
   'V': 0,
   'W': 0,
   'Y': 0}),
 ('P',
  {'-': 1.0,
   '

## At which positions are the sequences identical?
To answer this question, we will use a familiar [for loop](https://github.com/wilfredinni/python-cheatsheet#for-loops-and-the-range-function). When we encounter an `-` in the first element of the `position` tuple, this means that the human sequence had a `-` character at that position. `-` is not an amino acid, so we skip these positions and move on using the [continue statement](https://github.com/wilfredinni/python-cheatsheet#continue-statements).

In the `print` statement at the end of the cell, we also make use of [formatted strings](https://github.com/wilfredinni/python-cheatsheet#formatted-string-literals-or-f-strings-python-36) in Python 3.6.

In [167]:
# we want to keep track of which amino acid our
# "cursor" is on in the for loop
position_counter = 0

for position in hum_cat_pssm.pssm:
    
    # `position` is the 2-element tuple
    # let's give each element a useful name
    resi_in_human = position[0]
    resi_dict = position[1]
    
    # skip this position if it is a '-'
    # in the human sequence record
    if resi_in_human == '-':
        continue
    else:
        # increment the counter by 1
        position_counter += 1
    
    # if more than one instance of amino acid
    # `resi_in_human` was found at this position,
    # meaning that the cat homolog is the same amino acid
    if position[1][resi_in_human] > 1:
        print(f"cat and human are the same at position " +
              f"{position_counter}, which is amino acid {resi_in_human}")

cat and human are the same at position 16, which is amino acid G
cat and human are the same at position 47, which is amino acid A
cat and human are the same at position 82, which is amino acid Y
cat and human are the same at position 106, which is amino acid P
cat and human are the same at position 157, which is amino acid C
cat and human are the same at position 170, which is amino acid C
cat and human are the same at position 172, which is amino acid G
cat and human are the same at position 176, which is amino acid C
cat and human are the same at position 179, which is amino acid G
cat and human are the same at position 205, which is amino acid W
cat and human are the same at position 222, which is amino acid C
cat and human are the same at position 281, which is amino acid C
cat and human are the same at position 282, which is amino acid G
cat and human are the same at position 292, which is amino acid R
cat and human are the same at position 293, which is amino acid I
cat and human

To make sure our `position_counter` variable is working properly, let's double check that the length of the human sequence (without `-` characters) is indeed 529:

In [172]:
# position counter from the above for loop
print(f"the human sequence is {position_counter} amino acids long")

# calling len(str)
length_a_different_way = len(str(hum_cat_aln[0].seq).replace('-', ''))
print(f"the human sequence is {length_a_different_way} amino acids long")

the human sequence is 529 amino acids long
the human sequence is 529 amino acids long


We see that `position_counter` appears to be working as expected!

# ----- Code below is work in progress -----

60918

## Compute PSSM

In [ ]:
my_pssm = summary_align.pos_specific_score_matrix(consensus) # chars_to_ignore=["N"]
my_pssm

In [ ]:
my_pssm[1]

In [ ]:
replace_info = summary_align.replacement_dictionary(["G", "A"])
replace_info.keys()

In [14]:
replace_info[replace_info.keys()[0]]

NameError: name 'replace_info' is not defined

## Compute substitution and log odds matrix

In [15]:
from Bio import SubsMat
my_arm = SubsMat.SeqMat(replace_info)
my_arm

NameError: name 'replace_info' is not defined

In [16]:
my_lom = SubsMat.make_log_odds_matrix(my_arm)
my_lom

NameError: name 'my_arm' is not defined